In order to run this, you should have vector database `(vector_db_)` already stored as .zip. You can download it using the link
https://drive.google.com/drive/folders/1o2Y-BLNWNFT5zesysd2n3QZPtN0Jro3I?usp=drive_link

In [1]:
# After running this code the KERNEL needs to be restarted

!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

Run the code snippet below it has every library included.

In [2]:
import os
import pandas as pd
from transformers import pipeline

# Langchain modules
from langchain.document_loaders import UnstructuredFileLoader, PyPDFDirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TokenTextSplitter,
)
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq


# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"


# Download Sentence Transformers Embedding From Hugging Face
#embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/average_word_embeddings_komninos') #best
#embeddings = HuggingFaceEmbeddings() # Make sure you use the same embedders as you used to embed the original dataset into vector database.


/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/WordEmbeddings.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(os.path.join(i

There are two ways to read and predict in the model, Manual and Automatic

# Manual entry of vector database

In [2]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")


Extracting /content/auto_2001_new_embed_full.zip...
Extraction completed for auto_2001_new_embed_full.zip!


In [ ]:
# Retrieving the first vector database
persist_directory_1 = "/content/content/auto_book_1"
vectordb_1 = Chroma(
    persist_directory=persist_directory_1,
    embedding_function=embeddings
)

# Retrieving the second vector database
persist_directory_2 = "/content/content/dheer_auto_parts_db"
vectordb_2 = Chroma(
    persist_directory=persist_directory_2,
    embedding_function=embeddings
)
"""
# Retrieving the third vector database
persist_directory_3 = "/content/content/auto_book_2"
vectordb_3 = Chroma(
    persist_directory=persist_directory_3,
    embedding_function=embeddings
)
"""

# Create retrievers for both vector databases
retriever_1 = vectordb_1.as_retriever()
retriever_2 = vectordb_2.as_retriever()
#retriever_3 = vectordb_3.as_retriever()

# Define a combined retriever function
def combined_retriever(query):
    # Retrieve documents from both retrievers
    results_1 = retriever_1.get_relevant_documents(query)
    results_2 = retriever_2.get_relevant_documents(query)
    #results_3 = retriever_3.get_relevant_documents(query)

    # Combine results (you can sort or filter if needed)
    combined_results = results_1 + results_2 #+ results_3
    return combined_results

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.2-90b-vision-preview",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")


In [ ]:
# Create a prompt template
chatbot_prompt = """
You are an automotive parts assistant. When a user asks about their vehicle, you will refer to relevant documents and provide guidance in a concise, clear manner. Your goal is to stay under 1,000 tokens for each response, including all necessary details.

Here are the steps to follow:

1. **Identify the Vehicle and Parts**: Determine what part the user is asking about based on their question. If it’s unclear, ask clarifying questions to understand the model, make, and year of the vehicle.

2. **Provide Pricing**: Always provide the price for the part requested. If you don’t have the exact year of the vehicle the user asks about, provide the price for the earliest year available in your database and inform the user that you cannot help with the exact year requested. You should always mention the earliest year available, even if the part is not available for the requested year.

3. **Include Related Subcategories**: If the part requested falls under a category that has subcategories (e.g., "engine parts" has "fuel injectors", "oil filters", etc.), list those subcategories with their prices, if available.

4. **Mention Fluids**: If the part requested is related to fluids (e.g., oil, transmission fluid), also mention the fluids associated with the part and their availability/price.

5. **Maintain Chat Context**: Keep track of previous conversations and refer to them when necessary to provide consistent follow-up answers. For example, if the user has already asked about a part and later asks about another part from the same vehicle, refer back to previous details such as model, year, or part-related info.

6. **Structure the Response**:
   - Begin by acknowledging the vehicle type and confirming details, especially if the model year or make was mentioned.
   - Provide the price for the part.
   - List any relevant subcategories and related fluids.
   - If no exact match for the year is found, state the earliest year available in the database and explain the limitation.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}

"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.run(
            input_documents=combined_results,
            question=prompt  # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: I have 2011 toyota rav 4 and my brakes are not working

 **SBG**: I understand you're having issues with your 2011 Toyota RAV4's brakes. I've found some relevant information regarding brake performance. According to the document, "Some brakes stop well when they are cold." This might be related to your issue.

For a 2011 Toyota RAV4, the brake pads cost around $50-$70 per set. However, I would recommend checking the brake rotors as well, which cost around $200-$300 per set.

Some related subcategories for brake parts include:

- Brake pads ($50-$70 per set)
- Brake rotors ($200-$300 per set)
- Brake fluid (available, $10-$20 per liter)

Please note that these prices are estimates and may vary depending on the specific part and brand.

Additionally, I would like to ask if you have checked the brake fluid level and condition in your vehicle?

 **You**: waht about hydrolics ?

 **SBG**: I understand you're i

# Automatic entry of vector database


## Unziping

In [34]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")

Extracting /content/auto_book1_new_embed_full.zip...
Extraction completed for auto_book1_new_embed_full.zip!


## Reading the vector database

In [12]:
#!pip install chromadb
import os
from langchain.vectorstores import Chroma

# Directory containing vector databases
vector_db_dir = "/content/content/"
embedding_function = embeddings  # Ensure this is defined earlier in your code

# Initialize a list to store retrievers
retrievers = []

# Iterate through all subdirectories in the specified directory
for sub_dir in os.listdir(vector_db_dir):
    full_path = os.path.join(vector_db_dir, sub_dir)
    if os.path.isdir(full_path):  # Check if it's a directory
        print(f"Loading vector database from: {full_path}")

        # Load the vector database
        vectordb = Chroma(
            persist_directory=full_path,
            embedding_function=embedding_function
        )

        # Create a retriever and add it to the list
        retrievers.append(vectordb.as_retriever())

# Define a combined retriever function
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        # Retrieve documents from each retriever
        combined_results.extend(retriever.get_relevant_documents(query))

    return combined_results

# Example Usage
query = "What is the price of a fuel injector for 2001 toyota rav4 ?"
results = combined_retriever(query)
if results:
    print("Source document: ", results[0].metadata["source"])

# Print the number of documents retrieved
print(f"Retrieved {len(results)} documents.")

# Optionally print the source document (first result)



# Initialize the LLM
llm = ChatGroq(
    model="llama-3.2-90b-vision-preview",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="refine")


Loading vector database from: /content/content/auto_2000_new_embed_full
Loading vector database from: /content/content/.ipynb_checkpoints
Source document:  /content/all_data_2000.csv
Retrieved 4 documents.


## Chatbot finetuning and chatting

In [23]:
# Create a prompt template
chatbot_prompt = """
You are an intelligent and efficient **Automotive Parts and Fluids Assistant**. Your goal is to assist customers by identifying the most relevant parts and fluids for their vehicles based on their questions or descriptions. Always prioritize clarity, accuracy, and customer satisfaction. Follow these guidelines:
If a customer says something generic like, “My car isn’t starting,” the issue could be from multiple causes such as the alternator, battery, starter, oil leaks, etc. the chatbot would ask clarifying questions which are very helpful for initial inspection such as “Does the engine turn over?” or “Did you notice oil drips on your driveway?” or “Did the car stop after reaching 90 km/h?”. The chatbot would then provide an estimate for the mandatory "No Start" inspection and, based on the customer’s responses, suggest the top three most likely repair orders along with the total cost. The model would continuously optimize itself by learning from feedback given by auto service advisors on the repair orders.

On the auto parts side, the bot could easily provide customers with the parts they ask for specifically when customers are unsure of the exact name of a part, they can describe the parts and based on it the chatbot would give top 3 parts that matches with description given by customer. It would also show availability, different manufacturers and prices, stock levels, and warranty details in real-time.
**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.run(
            input_documents=combined_results,
            question=prompt  # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: 2000 Tyota rav4 brakes

 **SBG**: For your 2000 Toyota RAV4 with a 2.4L engine and automatic transmission, I found the drum brakes part.

Part Category: Braking System
Part Name: Drum Brakes
Part Manufacturer: Brembo
Part Number: BODRU1960
Price: $216.66

Would you like to know more about this part or check for other brake-related components, such as brake pads or rotors?

 **You**: manufacturer for the parts

 **SBG**: The new context is not relevant to the original question. The original question was about the manufacturer for the 2000 Toyota RAV4 drum brakes.

The original answer remains the same:

 For the 2000 Toyota RAV4 drum brakes, the part manufacturer is Brembo.

 **You**: exit
Chatbot: Goodbye!


# One for all
Just one retriever.


The code below stores previous chat and respond accordingly. Have to work on prompting






In [4]:
# Download Sentence Transformers Embedding From Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# Initialize embeddings, vector store, retriever, and LLM
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/average_word_embeddings_komninos')
persist_directory = "/content/content/auto_2000_new_embed_full"

# Initialize Chroma vector store
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

# Set up the retriever
retriever = vectordb.as_retriever()

# LLM from Groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

# Configuration for conversation buffer memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="result")

# Create the QA chain with memory
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    memory=memory  # Pass the memory here
)

# Interactive loop for querying
print("Welcome to the chatbot. Type 'exit' to stop.")
while True:
    query = input("You: ")

    if query.lower() == 'exit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Get the response from the QA chain
    response = qa_chain.invoke({"query": query})

    # Print the result
    print("Bot: ", response["result"])

    # Optionally print the source document (first result)
    if response.get("source_documents"):
        print("Source document: ", response["source_documents"][0].metadata["source"])

    # Debug: Show current conversation memory (optional)
    print("\nMemory so far:\n", memory.buffer)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/WordEmbeddings.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(os.path.join(i

Welcome to the chatbot. Type 'exit' to stop.
You: 2000 Audi A4 brakes
Bot:  I don't have information about the 2000 Audi A4 brakes. The provided context is about the 2000 Audi A3, not the A4.
Source document:  /content/all_data_2000.csv

Memory so far:
 [HumanMessage(content='2000 Audi A4 brakes'), AIMessage(content="I don't have information about the 2000 Audi A4 brakes. The provided context is about the 2000 Audi A3, not the A4.")]
You: 2000 Audi A3 brakes
Bot:  For a 2000 Audi A3, I found information on the following brake options:

1. Drum Brakes:
   - Engine Size: 1.8L
   - Part Manufacturer: Brembo
   - Part Number: BODRU8676
   - Price: $200.5

2. Disc Brakes:
   - Engine Size: 1.8L
   - Part Manufacturer: Brembo
   - Part Number: BODIS9113
   - Price: $212.68

3. Drum Brakes (different engine size):
   - Engine Size: 5.7L
   - Part Manufacturer: Brembo
   - Part Number: BODRU8570
   - Price: $208.68

Please note that the information provided does not include details on the flui

KeyboardInterrupt: Interrupted by user

Prompting

In [3]:
# Download Sentence Transformers Embedding From Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# Initialize embeddings, vector store, retriever, and LLM
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/average_word_embeddings_komninos')
persist_directory = "/content/content/auto_2000_new_embed_full"

# Initialize Chroma vector store
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

# Set up the retriever
retriever = vectordb.as_retriever()

# LLM from Groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)

# Configuration for conversation buffer memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="result")

# Define the custom prompt template
chatbot_prompt = """
You are an intelligent and efficient **Automotive Parts and Fluids Assistant**. Your goal is to assist customers by identifying the most relevant parts and fluids for their vehicles based on their questions or descriptions. Always prioritize clarity, accuracy, and customer satisfaction. Follow these guidelines:
- If a customer says something generic like, “My car isn’t starting,” the issue could be from multiple causes such as the alternator, battery, starter, oil leaks, etc. The chatbot would ask clarifying questions like “Does the engine turn over?” or “Did you notice oil drips on your driveway?”
- For parts, if a customer is unsure of the exact name of a part, they can describe the part and the chatbot should suggest top 3 parts that match the description. It will also provide availability, different manufacturers, prices, stock levels, and warranty details.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Create the QA chain with memory, prompt, and retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    memory=memory  # Pass the memory here
)

# Interactive loop for querying
print("Welcome to the chatbot. Type 'exit' to stop.")
while True:
    query = input("You: ")

    if query.lower() == 'exit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Create the prompt for the current conversation context
    prompt = chatbot_prompt.format(
        chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in memory.buffer]),
        user_input=query
    )

    # Get the response from the QA chain, passing the custom prompt as context
    response = qa_chain.invoke({"query": prompt})

    # Print the result
    print("Bot: ", response["result"])

    # Optionally print the source document (first result)
    if response.get("source_documents"):
        print("Source document: ", response["source_documents"][0].metadata["source"])

    # Debug: Show current conversation memory (optional)
    print("\nMemory so far:\n", memory.buffer)


Welcome to the chatbot. Type 'exit' to stop.
You: 2000 Audi A4 brakes
Bot:  I don't have information on a 2000 Audi A4. The data I have is for a 2000 Ford Focus. If you'd like to know more about the Ford Focus, I can try to help. Alternatively, you can provide more details about your Audi A4 brakes, such as the type of brake issue you're experiencing or the specific brake component you're looking for.
Source document:  /content/all_data_2000.csv

Memory so far:
 [HumanMessage(content="\nYou are an intelligent and efficient **Automotive Parts and Fluids Assistant**. Your goal is to assist customers by identifying the most relevant parts and fluids for their vehicles based on their questions or descriptions. Always prioritize clarity, accuracy, and customer satisfaction. Follow these guidelines:\n- If a customer says something generic like, “My car isn’t starting,” the issue could be from multiple causes such as the alternator, battery, starter, oil leaks, etc. The chatbot would ask clar

TypeError: 'HumanMessage' object is not subscriptable

Just for trying using one query

In [56]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/average_word_embeddings_komninos')
persist_directory = "/content/content/auto_2000_new_embed_full"
from langchain_chroma import Chroma
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)


# retriever
retriever = vectordb.as_retriever()
# llm from groq
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0
)
config = {"configurable": {"thread_id": "dheer123"}}
# create a qa chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    #config=config,

)
# invoke the qa chain and get a response for user query
query = "Prices for brakes of Toyota Rav4 2000"
response = qa_chain.invoke({"query": query})
print(response["result"])
print(response["source_documents"][0].metadata["source"])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/WordEmbeddings.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(os.path.join(i

For a 2000 Toyota RAV4, the prices for brakes are as follows:

- 1.8L engine, Automatic transmission:
  - Disc Brakes: $189.94
  - Drum Brakes: $211.32

- 2.4L engine, Automatic transmission:
  - Disc Brakes: $194.63
  - Drum Brakes: $216.66
/content/all_data_2000.csv


In [72]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# Directory containing the subdirectories for vector databases
vector_db_dir = "/content/content"
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/average_word_embeddings_komninos')

# Initialize a list to store retrievers
retrievers = []

# Iterate through all subdirectories in the specified directory
for sub_dir in os.listdir(vector_db_dir):
    full_path = os.path.join(vector_db_dir, sub_dir)
    if os.path.isdir(full_path):  # Check if it's a directory
        print(f"Loading vector database from: {full_path}")

        # Load the vector database
        vectordb = Chroma(
            persist_directory=full_path,
            embedding_function=embedding_function
        )

        # Create a retriever and add it to the list
        retrievers.append(vectordb.as_retriever())

# Define a combined retriever function
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        # Retrieve documents from each retriever
        combined_results.extend(retriever.get_relevant_documents(query))

    return combined_results

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.1-70b-versatile",  # Use the appropriate model as per your setup
    temperature=0
)

# Configuration for conversation buffer memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="result")

# Create the QA chain with memory and combined retriever
qa_chain = RetrievalQA.invoke(
    llm=llm,
    chain_type="stuff",
    retriever=combined_results,
    return_source_documents=True,
    memory=memory  # Pass the memory here
)

# Interactive loop for querying
print("Welcome to the chatbot. Type 'exit' to stop.")
while True:
    query = input("You: ")

    if query.lower() == 'exit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Get the response from the QA chain
    response = qa_chain.invoke({"query": query})

    # Print the result
    print("Bot: ", response["result"])

    # Optionally print the source document (first result)
    if response.get("source_documents"):
        print("Source document: ", response["source_documents"][0].metadata["source"])

    # Debug: Show current conversation memory (optional)
    print("\nMemory so far:\n", memory.buffer)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/WordEmbeddings.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(os.path.join(i

Loading vector database from: /content/content/auto_book_1_embed_full
Loading vector database from: /content/content/auto_2000_new_embed_full
Loading vector database from: /content/content/auto_2001_new_embed_full


NameError: name 'combined_results' is not defined

#

In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Add get_chat_history function

def get_chat_history(memory: ConversationBufferMemory) -> str:
    """Extract and format chat history from memory."""
    if memory and memory.chat_memory.messages:
        return "\n".join([
            f"{msg.type.capitalize()}: {msg.content}"
            for msg in memory.chat_memory.messages
        ])
    return ""


template = """Answer the question in your own words as truthfully as possible from the context given to you.
If you do not know the answer to the question, simply respond with "I don't know. Can you ask another question".
If questions are asked where there is no relevant context available, simply respond with "I don't know. Please ask a question relevant to the documents"
Context: {context}


{chat_history}
Human: {question}
Assistant:"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=template
)

# Create the custom chain
if llm is not None and vectordb is not None:
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=vectordb.as_retriever(), memory=memory,
        get_chat_history=get_chat_history, return_source_documents=True,
        combine_docs_chain_kwargs={'prompt': prompt})
else:
    logger.error("LLM or Vector Database not initialized")

# Chat on terminal
print("Welcome to the AI Chat! Type 'exit' to end the chat.")
while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break

        # Pass the user input to the chain
        response = chain({"question": user_input})
        print(f"Assistant: {response['answer']}")

        # Optional: Print source documents if needed
        if 'source_documents' in response:
            print("\n--- Sources ---")
            for doc in response['source_documents']:
                print(f"{doc.metadata['source']}: {doc.page_content[:200]}...")  # Adjust content display as needed
            print("----------------\n")
    except Exception as e:
        print(f"An error occurred: {e}")

Welcome to the AI Chat! Type 'exit' to end the chat.
You: 2000 toyota rav 4 brakes
An error occurred: 'list' object has no attribute 'chat_memory'
You: quit
Goodbye!


In [22]:
from langchain.memory import ConversationBufferMemory

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# Ensure this code works with your chain definition
if llm is not None and vectordb is not None:
    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        get_chat_history=get_chat_history,
        return_source_documents=True,
        combine_docs_chain_kwargs={'prompt': prompt}
    )
else:
    logger.error("LLM or Vector Database not initialized")

# Practice

In [ ]:
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing i

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser

In [ ]:
!pip install -qU langchain-groq

In [ ]:
import getpass
import os

os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192") #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